# Tax AI Assistant with LLM and RAG

Ever wished you had a tax expert on call to decode Germany's crazy-complicated tax laws? Yeah, me too! Today, we're going to build our very own AI tax assistant that can actually explain this stuff and answer our questions. Grab your coffee and let's get coding!

### Technical Approach

- **Model Selection:** The system utilizes Qwen2.5, a high-performing, multilingual open-source decoder-based LLM, chosen for its balance of performance and efficiency. Specifically, I experimented with the 1.5B parameter variant, which is compact enough to run on a single GPU while maintaining strong multilingual capabilities.

- **RAG Pipeline:**

    - **Vector Store:** FAISS for efficient similarity search over embedded legal texts.

    - **Embedding Model:** granite-embedding-278m-multilingual (ranked among the top multilingual models on Hugging Face’s Embedding Leaderboard), selected for its optimal trade-off between multilingual performance and hardware efficiency.

    - **Document Processing:** LangChain for chunking, embedding, and storing legal documents in a retrievable format.

This notebook documents the implementation, from setup to deployment, providing a reproducible framework for building domain-specific AI assistants.

Let’s begin by installing the required dependencies and importing the necessary modules.

In [1]:
%%capture
!pip install langchain langchain-community
!pip install faiss-cpu
!pip install huggingface_hub

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup
import requests
import re

## Acquiring Legal Text Data

To demonstrate our RAG pipeline, we will utilize publicly available German legal texts. As most German legislation is published online in accessible formats, we can easily obtain the required documents. For this implementation, we will focus on the Foreign Tax Law (Außensteuergesetz) as our use case, which governs taxation matters involving foreign entities and cross-border transactions.

The data acquisition process involves two key steps:

- Downloading the raw legal text from the official government website

- Extracting and structuring the content using BeautifulSoup for HTML parsing

In [3]:
url = "https://www.gesetze-im-internet.de/astg/BJNR117130972.html"
response = requests.get(url)
page_code = BeautifulSoup(response.text, 'html.parser')

## Legal Text Parsing and Context Preservation

The following function extracts complete law articles from the parsed web content. Maintaining full article context is critical for two reasons:

1. Legal provisions often contain interdependent clauses that require holistic understanding

2. Isolated sentences may lack the necessary legal nuance or qualifying conditions

By processing complete articles as discrete units, we ensure the LLM receives sufficient context for accurate interpretation and response generation.

In [4]:
def get_articles_from_page(soup):
    result = []

    for jnenbez in soup.find_all('span', class_='jnenbez'):
        jnnorm = jnenbez.find_parent('div', class_='jnnorm')
        if not jnnorm:
            continue

        # Find all jurAbsatz not under jnfussnote
        jur_absatz_divs = []
        for div in jnnorm.find_all('div', class_='jurAbsatz'):
            if not div.find_parent('div', class_='jnfussnote'):
                jur_absatz_divs.append(div)

        if not jur_absatz_divs:
            continue  # Skip jnenbez with no valid jurAbsatz

        # Extract header text from h3 containing jnenbez and jnentitel
        h3 = jnenbez.find_parent('h3')
        header = ''
        if h3:
            header = h3.get_text(separator=' ', strip=True)
            header = header.replace('\xa0', ' ')  # Replace non-breaking space with regular space
            header = re.sub(r'\s+', ' ', header).strip()

        # Process each jurAbsatz text
        jur_absatz_texts = []
        for idx, div in enumerate(jur_absatz_divs):
            text = div.get_text(separator=' ', strip=True)
            text = text.replace('\xa0', '')  # Remove all non-breaking spaces
            text = re.sub(r'\s+', ' ', text).strip()

            if idx == 0:
                text = f"{header}\n\n{text}" if header else text
            jur_absatz_texts.append(text)

        result.append(jur_absatz_texts)

    return result

## Document Chunking and Context Management

We will now process the parsed legal content by:

1. Applying the `RecursiveCharacterTextSplitter` from LangChain to divide the text into semantically coherent segments

1. Creating `Document` objects for each chunk while preserving metadata references to their source articles

This approach ensures that:

- Text segmentation maintains logical boundaries while adhering to token limits

- Each processed chunk retains access to its full article context when needed for RAG retrieval

- The original legal structure and relationships between provisions are preserved

Implementation notes:

- Chunk size optimized for model context windows

- Overlap configured to prevent loss of continuity between segments

In [5]:
# parse the page content and retrieve list of articles
articles = get_articles_from_page(page_code)

# Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30
)

search_chunks = []
for paragraphs in articles:
    parent_document = Document(page_content='\n\n'.join(paragraphs))
    for paragraph in paragraphs:
        small_chunks = text_splitter.split_text(paragraph)
        for chunk in small_chunks:
            search_chunks.append(Document(
                page_content=chunk,
                metadata={"parent": parent_document}
            ))

# examine the first 3 chunks
for i, chunk in enumerate(search_chunks[:3]):
    print(f"\n=== Chunk {i + 1} ===\n")
    print(chunk.page_content)


=== Chunk 1 ===

§ 1 Berichtigung von Einkünften

=== Chunk 2 ===

(1) Werden Einkünfte eines Steuerpflichtigen aus einer Geschäftsbeziehung zum Ausland mit einer ihm nahestehenden Person dadurch gemindert, dass er seiner Einkünfteermittlung andere Bedingungen, insbesondere Preise (Verrechnungspreise), zugrunde legt, als sie voneinander unabhängige Dritte unter

=== Chunk 3 ===

unabhängige Dritte unter gleichen oder vergleichbaren Verhältnissen vereinbart hätten (Fremdvergleichsgrundsatz), sind seine Einkünfte unbeschadet anderer Vorschriften so anzusetzen, wie sie unter den zwischen voneinander unabhängigen Dritten vereinbarten Bedingungen angefallen wären.


## Vector Embedding Generation and Indexing

Now we will store these Documents into our vector store FAISS and create the sentence embedding model that will compute the vector embeddings for these chunks.


In [7]:
# Create embeddings and store in FAISS
embedding_model = SentenceTransformerEmbeddings(model_name="ibm-granite/granite-embedding-278m-multilingual")
db = FAISS.from_documents(search_chunks, embedding_model)

## Document Similarity

Let's write a test query to see if FAISS really returns similar documents or not.

In [8]:
test_query = "wenn eine person an einer ausländische Körperschaft beteiligt ist, werden die Einfünfte aus dieser Körperschaft steuerpflichtig?"
docs = db.similarity_search(test_query, k=3)
print("\n\n".join([doc.page_content for doc in docs]))

(1) Erhält der Steuerpflichtige aus der Beteiligung an einer ausländischen Gesellschaft, für die Hinzurechnungsbeträge nach § 10 Absatz 2 bei ihm der Einkommen- oder Körperschaftsteuer unterlegen haben, Bezüge im Sinne des 1. § 20 Absatz 1 Nummer 1 des Einkommensteuergesetzes, 2. § 20 Absatz 1

ausländischen Gesellschaft, Beteiligungsverhältnisse und Identifikationsmerkmale der an der ausländischen Gesellschaft Beteiligten. Das zuständige Finanzamt kann in den Fällen des Satzes 2 die Abgabe einer Erklärung nach Satz 1 verlangen. Die Verpflichtungen nach diesem Absatz können durch die

wenn sie von den unbeschränkt Steuerpflichtigen, die gemäß § 7 an der ausländischen Gesellschaft beteiligt sind, unmittelbar bezogen worden wären, und c) die Vermietung oder Verpachtung von beweglichen Sachen, es sei denn, der Steuerpflichtige weist nach, dass die ausländische Gesellschaft einen


The returned documents are very similar and relevant to the query. That's a good start.

Below we print the full article to which each chunk belongs.

## Model Initialization

We now initialize the 'Qwen2.5-1.5B-Instruct' model and its tokenizer. This 'Instruct' model is a version of Qwen2.5-1.5B finetuned on instruction following.

In [10]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(model.device)

cuda:0


## Query Processing and Response Generation

Below is a function to process the user prompt, tokenizes it, calls the `model.generate()` method, and decodes the output and returns it as string.

In [11]:
def generate_answer(prompt):
    messages = [
        {"role": "system", "content": "You are Qwen. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=.6,
        top_p=.9
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

### Contextual Query Processing
The function processes user queries by first retrieving semantically similar legal documents from the FAISS index. It then extracts the complete parent articles and prepends them as contextual background to the original question.

### Transparency and Validation
The system displays the complete prompt with retrieved context prior to generation, enabling verification of context relevance and ensuring responses remain properly grounded in legal provisions.

In [21]:
def rag_query(user_query):
    # Retrieve top-k relevant documents
    matches = db.similarity_search(user_query, k=2)
    # retrieve the paranet articles to use it as context
    parent_paragraphs = list({doc.metadata['parent'].page_content for doc in matches})
    context = "\n\n".join(parent_paragraphs)

    # Combine context with query
    prompt = f"""
## Instruction:

Answer this legal question about the German tax law based on the context below.

## LEGAL CONTEXT:

{context}

## USER QUERY:

{user_query}

## MODEL RESPONSE:
    """

    print(prompt)

    return generate_answer(prompt)

## The Moment of Truth

Now let's ask questions and see if the returned answer is correct.

### First Question

I begin by asking a question about the tax liability in case of involvement in foreign corporations that brings income to the person.

In [22]:
query = "wenn eine person an einer ausländische Körperschaft beteiligt ist, werden die Einfünfte aus dieser Körperschaft steuerpflichtig sein?"
response = rag_query(query)
print(response)


## Instruction:

Answer this legal question about the German tax law based on the context below.

## LEGAL CONTEXT:

§ 11 Kürzungsbetrag bei Beteiligung an ausländischer Gesellschaft

(1) Erhält der Steuerpflichtige aus der Beteiligung an einer ausländischen Gesellschaft, für die Hinzurechnungsbeträge nach § 10 Absatz 2 bei ihm der Einkommen- oder Körperschaftsteuer unterlegen haben, Bezüge im Sinne des 1. § 20 Absatz 1 Nummer 1 des Einkommensteuergesetzes, 2. § 20 Absatz 1 Nummer 3 des Einkommensteuergesetzes in Verbindung mit § 16 Absatz 1 Nummer 1 des Investmentsteuergesetzes oder 3. § 20 Absatz 1 Nummer 3a des Einkommensteuergesetzes in Verbindung mit § 34 Absatz 1 Nummer 1 des Investmentsteuergesetzes, ist bei der Ermittlung der Summe der Einkünfte ein Kürzungsbetrag nach Absatz 2 abzuziehen; im Rahmen des § 32d des Einkommensteuergesetzes ist dieser bei der Ermittlung der Summe der Kapitalerträge abzuziehen. Entsprechendes gilt für Bezüge des Steuerpflichtigen im Sinne des Satze

**********************************************************************************                    
Great job. The context is relevant to the question and the model's answer is correct and is based on the context.

### Second Question

Let's test it again with another question. This next question is related to an article in the law that discusses the tax liabilty in case of moving out of the country and living in a low-tax country.

In [23]:
query = "wenn man sein Wohnsitz in einem niedrigbesteuernden Land wechselt und hat man Einkünften in Deutschland, bleibt man weiter steuerpflichtig?"
response = rag_query(query)
print(response)


## Instruction:

Answer this legal question about the German tax law based on the context below.

## LEGAL CONTEXT:

§ 10 Hinzurechnungsbetrag

(1) Die nach § 7 Absatz 1 steuerpflichtigen Einkünfte sind bei dem Steuerpflichtigen als Hinzurechnungsbetrag anzusetzen. Ergibt sich ein negativer Betrag, so entfällt die Hinzurechnung.

(2) Der Hinzurechnungsbetrag gehört zu den Einkünften im Sinne des § 20 Absatz 1 Nummer 1 des Einkommensteuergesetzes und gilt in dem Veranlagungszeitraum als zugeflossen, in dem das maßgebende Wirtschaftsjahr der ausländischen Gesellschaft endet. Gehören Anteile an der ausländischen Gesellschaft zu einem Betriebsvermögen, so gehört der Hinzurechnungsbetrag zu den Einkünften aus Gewerbebetrieb, aus Land- und Forstwirtschaft oder aus selbständiger Arbeit und erhöht den nach dem Einkommen- oder Körperschaftsteuergesetz ermittelten Gewinn des Betriebs für das Wirtschaftsjahr, in dem das Wirtschaftsjahr der ausländischen Gesellschaft endet. Sind dem Steuerpflicht

**********************************************************************************
The model response to question is correct but the details given in the response are not so accurate.

### Third Question

Let's make our last test and ask one more question. In the next question I ask if Germans will continue to be tax liable after moving out of Germany if they don't have any economic interests in Germany (such as a business or some income).

In [24]:
query = "bleiben deutsche Staatsbürger weiter Steuerpflichtig nach dem Auszug ins Ausland, wenn sie keine wirtschaftliche Interessen in Deutschland haben (keine Unternehmen oder Einkünfte)?"
response = rag_query(query)
print(response)


## Instruction:

Answer this legal question about the German tax law based on the context below.

## LEGAL CONTEXT:

§ 10 Hinzurechnungsbetrag

(1) Die nach § 7 Absatz 1 steuerpflichtigen Einkünfte sind bei dem Steuerpflichtigen als Hinzurechnungsbetrag anzusetzen. Ergibt sich ein negativer Betrag, so entfällt die Hinzurechnung.

(2) Der Hinzurechnungsbetrag gehört zu den Einkünften im Sinne des § 20 Absatz 1 Nummer 1 des Einkommensteuergesetzes und gilt in dem Veranlagungszeitraum als zugeflossen, in dem das maßgebende Wirtschaftsjahr der ausländischen Gesellschaft endet. Gehören Anteile an der ausländischen Gesellschaft zu einem Betriebsvermögen, so gehört der Hinzurechnungsbetrag zu den Einkünften aus Gewerbebetrieb, aus Land- und Forstwirtschaft oder aus selbständiger Arbeit und erhöht den nach dem Einkommen- oder Körperschaftsteuergesetz ermittelten Gewinn des Betriebs für das Wirtschaftsjahr, in dem das Wirtschaftsjahr der ausländischen Gesellschaft endet. Sind dem Steuerpflicht

**********************************************************************************

**The answer this time is wrong and contradicts the given context. It seems like the model was not able to reason properly about that question.**

**********************************************************************************

## Conclusion and Future Work



The model performs reasonably well given its small size (1.5B parameters). However, there is still room for improvement. The model is likely not trained on enough legal data and it would be better if we finetune the model on legal data.

### Why finetuning helps:

* **Domain Adaptation:** Legal texts require understanding of jargon, nested logic, and cross-referenced articles. Fine-tuning aligns the model’s embeddings with legal semantics.

* **Improved Context Utilization:** Even with RAG, smaller models may fail to properly prioritize or synthesize retrieved legal context. Fine-tuning teaches the model to focus on key phrases (e.g., "Article 12", "tax liability") and structure answers appropriately.

* **Reduced Hallucinations:** Models untrained on legal data may "guess" based on general knowledge, leading to inaccuracies. Fine-tuning anchors responses to legal terminology and logic.

Below are some ideas for further improvements.

### Training Tasks for Legal Fine-Tuning

### 1. Legal Corpus Pre-Training

- **Objective:** Adapt the model’s base knowledge to legal language.

- **Data:** Use legal texts (statutes, court rulings, tax codes, contracts) from the target jurisdiction.

- **Tasks:**

    - **Causal Language Modeling:** Train the model to predict the next token in legal documents (helps with coherence in generating citations).

### 2. Question-Answering (QA) Fine-Tuning

- **Objective:** Teach the model to answer questions using legal references.

- **Data:**

    - **Synthetic QA Pairs:** Generate questions and answers from legal texts (e.g., "What is the penalty for late tax filing under Article 5?" → "Article 5 states...").

    - **Legal Exams/Textbooks:** Use existing QA datasets (e.g., Bar exam questions, legal textbooks).

    - **RAG-Style Training:** Provide "context" (retrieved law articles) and ask the model to answer questions based on it.

- **Tasks:**

    - **Extractive QA:** Identify spans of text in the context that answer the question (e.g., exact articles or clauses).

    - **Summarization:** Condense legal text into concise answers (e.g., "Summarize the tax exemptions in Article 3").

### 3. Citation and Reference Training

- **Objective:** Ensure the model cites correct articles and avoids contradictions.

- **Data:** Examples where answers must reference specific laws (e.g., "Under [Article 12], taxpayers must...").

- **Tasks:**

    - **Prompt Engineering:** Use templates like, "Based on [Article X], [explanation]."

    - **Negative Examples:** Penalize the model for citing irrelevant articles during training.

